In [39]:
#| default_exp datasets

In [40]:
# Install the required modules to run this notebook

# Uninstall current version of torch and related packages and install teh correct version to build mini.ai module in the next step
#!pip uninstall -y torch torchvision torchaudio torchtext fastai torchdata
!pip install torch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1 torchtext==0.14.1 fastai==2.7.11
!pip install torchdata==0.5.1
!pip install nbdev

# Install the miniai module
!pip install -Uqq git+https://github.com/fastai/course22p2



  Preparing metadata (setup.py) ... done


In [41]:
#|export
from __future__ import annotations
import math,numpy as np,matplotlib.pyplot as plt
from operator import itemgetter
from itertools import zip_longest
import fastcore.all as fc

from torch.utils.data import default_collate

from miniai.training import *

In [42]:
import logging,pickle,gzip,os,time,shutil,torch,matplotlib as mpl
from pathlib import Path

from torch import tensor,nn,optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
from datasets import load_dataset,load_dataset_builder

import torchvision.transforms.functional as TF
from fastcore.test import test_close

In [43]:
torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)
torch.manual_seed(1)
mpl.rcParams['image.cmap'] = 'gray'

In [44]:
logging.disable(logging.WARNING)

In [94]:
!pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
aiohttp                          3.9.1
aiosignal                        1.3.1
alabaster                        0.7.13
albumentations                   1.3.1
altair                           4.2.2
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array-record                     0.5.0
arviz                            0.15.1
astropy                          5.3.4
asttokens                        2.4.1
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.0
attrs                            23.2.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.14.0
backcall                         0.2.0
beautifulsoup4                   4.11.2
bi

## Hugging Face Datasets

In [45]:
name = "fashion_mnist"
ds_builder = load_dataset_builder(name)
print(ds_builder.info.description)

In [46]:
ds_builder.info

DatasetInfo(description='', citation='', homepage='', license='', features={'image': Image(decode=True, id=None), 'label': ClassLabel(names=['T - shirt / top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'], id=None)}, post_processed=None, supervised_keys=None, task_templates=None, builder_name='parquet', dataset_name='fashion_mnist', config_name='fashion_mnist', version=1.0.0, splits={'train': SplitInfo(name='train', num_bytes=31304107, num_examples=60000, shard_lengths=None, dataset_name='fashion_mnist'), 'test': SplitInfo(name='test', num_bytes=5235060, num_examples=10000, shard_lengths=None, dataset_name='fashion_mnist')}, download_checksums={'hf://datasets/fashion_mnist@fae1c0e55b2a1b26e007544c451deead66388cf1/fashion_mnist/train/0000.parquet': {'num_bytes': 30931277, 'checksum': None}, 'hf://datasets/fashion_mnist@fae1c0e55b2a1b26e007544c451deead66388cf1/fashion_mnist/test/0000.parquet': {'num_bytes': 5175617, 'checksum': None}}, downlo

In [47]:
ds_builder.info.features

{'image': Image(decode=True, id=None),
 'label': ClassLabel(names=['T - shirt / top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'], id=None)}

In [48]:
ds_builder.info.splits

{'train': SplitInfo(name='train', num_bytes=31304107, num_examples=60000, shard_lengths=None, dataset_name='fashion_mnist'),
 'test': SplitInfo(name='test', num_bytes=5235060, num_examples=10000, shard_lengths=None, dataset_name='fashion_mnist')}

In [49]:
dsd = load_dataset(name)
dsd

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 60000
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 10000
    })
})

In [50]:
train,test = dsd['train'],dsd['test']
train[0]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
 'label': 9}

In [51]:
x,y = 'image','label'
img = train[0][x]
img

In [52]:
xb = train[:5][x]
yb = train[:5][y]
yb

[9, 0, 0, 3, 0]

In [98]:
featy = train.features[y]
featy

ClassLabel(names=['T - shirt / top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'], id=None)

In [54]:
featy.int2str(yb)

['Ankle boot',
 'T - shirt / top',
 'T - shirt / top',
 'Dress',
 'T - shirt / top']

In [55]:
train['label'][:5]

[9, 0, 0, 3, 0]

In [92]:
def collate_fn(b):
    import pdb; pdb.set_trace()
    return {x:torch.stack([TF.to_tensor(o[x]) for o in b]),
            y:tensor([o[y] for o in b])}

In [93]:
dl = DataLoader(train, collate_fn=collate_fn, batch_size=16)
b = next(iter(dl))
b[x].shape,b[y]

> <ipython-input-92-aec7666d7ce7>(3)collate_fn()
      1 def collate_fn(b):
      2     import pdb; pdb.set_trace()
----> 3     return {x:torch.stack([TF.to_tensor(o[x]) for o in b]),
      4             y:tensor([o[y] for o in b])}

ipdb> p b
[{'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28 at 0x7CD19F653A00>, 'label': 9}, {'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28 at 0x7CD19F651630>, 'label': 0}, {'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28 at 0x7CD19F651660>, 'label': 0}, {'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28 at 0x7CD19F6522C0>, 'label': 3}, {'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28 at 0x7CD19F651690>, 'label': 0}, {'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28 at 0x7CD19F652DD0>, 'label': 2}, {'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28 at 0x7CD19F650E80>, 'label': 7}, {'image': <PIL.PngImagePlugin.PngImageFile image mode=L

In [57]:
def transforms(b):
    # import pdb; pdb.set_trace()
    b[x] = [torch.flatten(TF.to_tensor(o)) for o in b[x]]
    return b

In [58]:
tds = train.with_transform(transforms)
dl = DataLoader(tds, batch_size=16)
b = next(iter(dl))
type(b) # dict
type(b[x]) # torch.Tensor
type(b[y]) # torch.Tensor
b[x].shape,b[y]

(torch.Size([16, 784]),
 tensor([9, 0, 0, 3, 0, 2, 7, 2, 5, 5, 0, 9, 5, 5, 7, 9]))

In [59]:
# Train using the model we created earlier but with tweaks

m = 784
nh = 100
lr = 1
final_activiations = 1000
def get_model():
    model = nn.Sequential(nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh,final_activiations))
    return model, optim.SGD(model.parameters(), lr=lr)

model, opt = get_model()


bs=50

train_ds = train.with_transform(transforms)
train_dl = DataLoader(train_ds, batch_size=bs)

test_ds = test.with_transform(transforms)
test_dl = DataLoader(test_ds, batch_size=bs*2)

loss_func = F.cross_entropy

# This will fail since the data returned by the DataLoader iterator is batch dictionary instead of batch tuple
#%time loss,acc = fit(5, model, loss_func, opt, train_dl, test_dl)

In [60]:
def _transformi(b): b[x] = [torch.flatten(TF.to_tensor(o)) for o in b[x]]

In [61]:
#|export
def inplace(f):
    def _f(b):
        f(b)
        return b
    return _f

In [62]:
transformi = inplace(_transformi)

In [63]:
from datasets import load_dataset, Dataset

# Create new HF DataSet with transforms.
# Our HF DataSet is a dictionary with keys images and labels
ds_with_transforms = train.with_transform(transformi)
# train is the fashion_mnist Dataset we loaded via load_dataset(..)["train"]
assert isinstance(ds_with_transforms, Dataset)

# training dataset for fashion_mnist has 60K records
assert len(ds_with_transforms) == 60000

# Accewssing the first element in Dataset will call __getitem__ which should call our transform that converts images into tensors
r = ds_with_transforms[0]
# Should return a dictionary with key x/images and value of type tensor.
#It is a tensors instead of list of PIL.PngImagePlugin.PngImageFile because transform function got called which converted image to tensor value
assert isinstance(r, dict)
assert isinstance(r[x], torch.Tensor)
# The labels/y had no transformation done to it, so it should be just a single scaler not a tensor
assert isinstance(r[y], int)

r[x].shape,r[y] # (torch.Size([784]), 9)


# Access the Dataset without transforms and you will see that the image is of type PngImage instead of tensor
r_no_transform = train[0]
assert isinstance(r_no_transform, dict)
type(r_no_transform[x]) # PIL.PngImagePlugin.PngImageFile

PIL.PngImagePlugin.PngImageFile

In [85]:
@inplace
def transformi(b):
  b[x] = [torch.flatten(TF.to_tensor(o)) for o in b[x]]

In [86]:
tdsf = train.with_transform(transformi)
r = tdsf[0]
r[x].shape,r[y]

(torch.Size([784]), 9)

In [66]:
d = dict(a=1,b=2,c=3)
ig = itemgetter('a','c')
ig(d)

(1, 3)

In [67]:
class D:
    def __getitem__(self, k): return 1 if k=='a' else 2 if k=='b' else 3

In [68]:
d = D()
ig(d)

(1, 3)

In [69]:
list(tdsf.features)

['image', 'label']

In [72]:
batch = dict(a=[1],b=[2]), dict(a=[3],b=[4])
default_collate(batch)

{'a': [tensor([1, 3])], 'b': [tensor([2, 4])]}

In [89]:
#|export
def collate_dict(ds):
    get = itemgetter(*ds.features)
    def _f(b):
      return get(default_collate(b))
    return _f

In [90]:
tdsf = train.with_transform(transformi)
dlf = DataLoader(tdsf, batch_size=4, collate_fn=collate_dict(tdsf))
xb,yb = next(iter(dlf))
xb.shape,yb

(torch.Size([4, 784]), tensor([9, 0, 0, 3]))

In [91]:
# Train using the model we created earlier but with tweaks

m = 784
nh = 100
lr = 1
final_activiations = 1000
def get_model():
    model = nn.Sequential(nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh,final_activiations))
    return model, optim.SGD(model.parameters(), lr=lr)

model, opt = get_model()


bs=50

train_ds = train.with_transform(transforms)
train_dl = DataLoader(train_ds, batch_size=bs, collate_fn=collate_dict(train_ds))

test_ds = test.with_transform(transforms)
test_dl = DataLoader(test_ds, batch_size=bs*2, collate_fn=collate_dict(test_ds))

loss_func = F.cross_entropy

# %time loss,acc = fit(5, model, loss_func, opt, train_dl, test_dl)

0 2.315779376029968 0.10029999978840351
1 2.310964686870575 0.10059999991208315
2 2.3099112939834594 0.09999999970197677
3 2.308979325294495 0.09999999970197677
4 2.3085274696350098 0.09999999970197677
CPU times: user 1min 13s, sys: 0 ns, total: 1min 13s
Wall time: 1min 14s


## Plotting images

In [ ]:
b = next(iter(dl))
xb = b['image']
img = xb[0]
plt.imshow(img[0]);

In [ ]:
#|export
@fc.delegates(plt.Axes.imshow)
def show_image(im, ax=None, figsize=None, title=None, noframe=True, **kwargs):
    "Show a PIL or PyTorch image on `ax`."
    if fc.hasattrs(im, ('cpu','permute','detach')):
        im = im.detach().cpu()
        if len(im.shape)==3 and im.shape[0]<5: im=im.permute(1,2,0)
    elif not isinstance(im,np.ndarray): im=np.array(im)
    if im.shape[-1]==1: im=im[...,0]
    if ax is None: _,ax = plt.subplots(figsize=figsize)
    ax.imshow(im, **kwargs)
    if title is not None: ax.set_title(title)
    ax.set_xticks([])
    ax.set_yticks([])
    if noframe: ax.axis('off')
    return ax

In [ ]:
help(show_image)

In [ ]:
show_image(img, figsize=(2,2));

In [ ]:
fig,axs = plt.subplots(1,2)
show_image(img, axs[0])
show_image(xb[1], axs[1]);

In [ ]:
#|export
@fc.delegates(plt.subplots, keep=True)
def subplots(
    nrows:int=1, # Number of rows in returned axes grid
    ncols:int=1, # Number of columns in returned axes grid
    figsize:tuple=None, # Width, height in inches of the returned figure
    imsize:int=3, # Size (in inches) of images that will be displayed in the returned figure
    suptitle:str=None, # Title to be set to returned figure
    **kwargs
): # fig and axs
    "A figure and set of subplots to display images of `imsize` inches"
    if figsize is None: figsize=(ncols*imsize, nrows*imsize)
    fig,ax = plt.subplots(nrows, ncols, figsize=figsize, **kwargs)
    if suptitle is not None: fig.suptitle(suptitle)
    if nrows*ncols==1: ax = np.array([ax])
    return fig,ax

In [ ]:
import nbdev;
from nbdev.showdoc import show_doc

In [ ]:
show_doc(subplots)

In [ ]:
fig,axs = subplots(3,3, imsize=1)
imgs = xb[:8]
for ax,img in zip(axs.flat,imgs): show_image(img, ax)

In [ ]:
#|export
@fc.delegates(subplots)
def get_grid(
    n:int, # Number of axes
    nrows:int=None, # Number of rows, defaulting to `int(math.sqrt(n))`
    ncols:int=None, # Number of columns, defaulting to `ceil(n/rows)`
    title:str=None, # If passed, title set to the figure
    weight:str='bold', # Title font weight
    size:int=14, # Title font size
    **kwargs,
): # fig and axs
    "Return a grid of `n` axes, `rows` by `cols`"
    if nrows: ncols = ncols or int(np.floor(n/nrows))
    elif ncols: nrows = nrows or int(np.ceil(n/ncols))
    else:
        nrows = int(math.sqrt(n))
        ncols = int(np.floor(n/nrows))
    fig,axs = subplots(nrows, ncols, **kwargs)
    for i in range(n, nrows*ncols): axs.flat[i].set_axis_off()
    if title is not None: fig.suptitle(title, weight=weight, size=size)
    return fig,axs

In [ ]:
fig,axs = get_grid(8, nrows=3, imsize=1)
for ax,img in zip(axs.flat,imgs): show_image(img, ax)

In [ ]:
#|export
@fc.delegates(subplots)
def show_images(ims:list, # Images to show
                nrows:int|None=None, # Number of rows in grid
                ncols:int|None=None, # Number of columns in grid (auto-calculated if None)
                titles:list|None=None, # Optional list of titles for each image
                **kwargs):
    "Show all images `ims` as subplots with `rows` using `titles`"
    axs = get_grid(len(ims), nrows, ncols, **kwargs)[1].flat
    for im,t,ax in zip_longest(ims, titles or [], axs): show_image(im, ax=ax, title=t)

In [ ]:
yb = b['label']
lbls = yb[:8]

In [ ]:
names = "Top Trouser Pullover Dress Coat Sandal Shirt Sneaker Bag Boot".split()
titles = itemgetter(*lbls)(names)
' '.join(titles)

In [ ]:
show_images(imgs, imsize=1.7, titles=titles)

In [ ]:
#|export
class DataLoaders:
    def __init__(self, *dls): self.train,self.valid = dls[:2]

    @classmethod
    def from_dd(cls, dd, batch_size, as_tuple=True, **kwargs):
        f = collate_dict(dd['train'])
        return cls(*get_dls(*dd.values(), bs=batch_size, collate_fn=f, **kwargs))

## Export -

In [ ]:
import nbdev; nbdev.nbdev_export()